In [ ]:
#''''''denoising network''''''


import numpy as np
from sklearn.model_selection import train_test_split

# === Load data ===
# Load training signals contaminated with 0dB noise
x_train = np.load('C:/AE_noise_localization/x_train_0dB.npy')  
# Load corresponding clean training signals
y_train = np.load('C:/AE_noise_localization/X_train.npy')       

# Load validation signals contaminated with 0dB noise
x_val = np.load('C:/AE_noise_localization/x_val_0dB.npy') 
# Load corresponding clean validation signals
y_val = np.load('C:/AE_noise_localization/X_val.npy') 


x_train = x_train[..., np.newaxis]  # (N, 1024, 1)
y_train = y_train[..., np.newaxis]

x_val = x_val[..., np.newaxis]  # (N, 1024, 1)
y_val = y_val[..., np.newaxis]


from tensorflow.keras import layers, models

# === Denoising network ===
def conv_block(x, filters, kernel_size=3, activation='relu'):
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.Activation(activation)(x)
    return x

def encoder_block(x, filters):
    x = conv_block(x, filters)
    p = layers.MaxPooling1D(pool_size=2)(x)
    return x, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling1D(size=2)(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

def build_unet_1d(input_length=1024):
    inputs = layers.Input(shape=(input_length, 1))
    e1, p1 = encoder_block(inputs, 32)
    e2, p2 = encoder_block(p1, 64)
    e3, p3 = encoder_block(p2, 128)
    e4, p4 = encoder_block(p3, 256)
    e5, p5 = encoder_block(p4, 512)
    e6, p6 = encoder_block(p5, 1024)
    b = conv_block(p6, 1024)
    d6 = decoder_block(b, e6, 1024)
    d5 = decoder_block(d6, e5, 256)
    d4 = decoder_block(d5, e4, 128)
    d3 = decoder_block(d4, e3, 64)
    d2 = decoder_block(d3, e2, 32)
    d1 = decoder_block(d2, e1, 32)
    outputs = layers.Conv1D(1, kernel_size=1, activation='linear')(d1)
    return models.Model(inputs=inputs, outputs=outputs)


# Build the model
model = build_unet_1d(input_length=1024)
model.summary()

from tensorflow.keras.callbacks import EarlyStopping

# === Compile the model ===
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# === Set up EarlyStopping (not used now) ===
from tensorflow.keras.callbacks import ModelCheckpoint

# === Set model saving path ===
checkpoint_path = 'C:/localization/denoise_0dB_opt_code.h5'

# === Save only the model with the lowest val_loss ===
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,  # Save the whole model (structure + weights)
    verbose=1
)

# === Train for 100 epochs (currently set to 1) ===
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=200,
    callbacks=[checkpoint],  # EarlyStopping not used
    verbose=1
)


# Load test signals contaminated with 0dB noise
x_test = np.load('C:/AE_noise_localization/x_test_0dB.npy')
# Load corresponding clean test signals
y_test = np.load('C:/AE_noise_localization/X_test.npy')

y_pred = model.predict(x_test, batch_size=2000)


y_pred = y_pred.squeeze()

def compute_snr(clean, denoised):
    signal_power = np.mean(clean ** 2)
    noise_power = np.mean((clean - denoised) ** 2)
    if noise_power == 0:
        return np.inf
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

snr_value = compute_snr(y_test, y_pred)
print(f"Test SNR: {snr_value:.2f} dB")


In [ ]:
#''''''localization network''''''



import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

def euclidean_distance_loss(y_true, y_pred):
    return tf.reduce_mean(tf.norm(y_true - y_pred, axis=-1))

# Load input waveforms for the training set
X_train = np.load('C:/AE_noise_localization/X_train.npy')
# Load the coordinates (labels) corresponding to the training signals
y_train = np.load('C:/AE_noise_localization/y_train_lable.npy')

# Load input waveforms for the validation set
X_val = np.load('C:/AE_noise_localization/X_val.npy')
# Load the coordinates (labels) corresponding to the validation signals
y_val = np.load('C:/AE_noise_localization/y_val_lable.npy')


inputs = layers.Input(shape=(1024,))

x = layers.Dense(1024, activation='relu')(inputs)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)

x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)

x = layers.Dense(1024, activation='relu')(x)

outputs = layers.Dense(2)(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss=euclidean_distance_loss)
model.summary()

# Checkpoint callback
model_path = os.path.join(save_path, 'best_model_OPT.h5')
checkpoint = ModelCheckpoint(
    filepath=model_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=2000,
    batch_size=3000,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint]
)

# Load the best model after training
best_model = load_model(
    model_path, 
    custom_objects={'euclidean_distance_loss': euclidean_distance_loss}
)
# Load input waveforms for the test set
X_test = np.load('C:/AE_noise_localization/X_test.npy')
# Load the coordinates (labels) corresponding to the test signals
y_test = np.load('C:/AE_noise_localization/y_test_lable.npy')

best_model = load_model(
    model_path, 
    custom_objects={'euclidean_distance_loss': euclidean_distance_loss}
)

# Evaluation on test set
test_loss = best_model.evaluate(X_test, y_test)
print("Mean Euclidean distance on the test set:", test_loss)
